<a href="https://colab.research.google.com/github/weathon/wild-fire/blob/main/WildFire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
# Dataset Used
# https://www.kaggle.com/datasets/brsdincer/wildfire-detection-image-data

In [ ]:
!unzip archive.zip

Archive:  archive.zip
replace forest_fire/Testing/fire/abc169.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install torchinfo

In [ ]:
import torch, torchinfo

class Block(torch.nn.Module):
  def __init__(self, i, o):
    super().__init__()
    self.conv1 = torch.nn.Conv2d(i, o, (3, 3), padding=1)
    self.pro1 = torch.nn.Conv2d(i, o, (1, 1))
    self.conv2 = torch.nn.Conv2d(o, o, (3, 3), 2)
    self.relu = torch.nn.ReLU()


  def forward(self, img):
    x = self.relu(self.conv1(img)) + self.pro1(img)
    return self.relu(self.conv2(x))

torchinfo.summary(Block(3, 16), (1,3,250,250))

In [ ]:
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.block1 = Block(3, 16)
    self.block2 = Block(16, 32)
    self.block3 = Block(32, 64)
    self.block4 = Block(64, 128)
    self.block5 = Block(128, 256)
    self.f = torch.nn.Flatten()
    self.FC1 = torch.nn.Linear(9216, 1024)
    self.relu1 = torch.nn.ReLU()
    self.FC2 = torch.nn.Linear(1024, 16)
    self.relu2 = torch.nn.ReLU()
    self.FC3 = torch.nn.Linear(16, 1)

  def forward(self, x):
    x =  self.f(self.block5(self.block4(self.block3(self.block2(self.block1(x))))))
    return self.FC3(self.relu2(self.FC2(self.relu1(self.FC1(x)))))


torchinfo.summary(Model(), (1024,3,250,250))

In [ ]:
images = []
tags = []

import cv2, os
for i in os.listdir("/content/forest_fire/Training and Validation/fire"):
  # https://stackoverflow.com/questions/50963283/imshow-doesnt-need-convert-from-bgr-to-rgb
  images.append(cv2.cvtColor(cv2.imread("/content/forest_fire/Training and Validation/fire/"+i), cv2.COLOR_BGR2RGB))
  tags.append(1)

for i in os.listdir("/content/forest_fire/Training and Validation/nofire"):
  images.append(cv2.cvtColor(cv2.imread("/content/forest_fire/Training and Validation/nofire/"+i), cv2.COLOR_BGR2RGB))
  tags.append(0)

In [ ]:
import pylab
pylab.imshow(images[0])
pylab.title(tags[0])
pylab.show()

pylab.imshow(images[-1])
pylab.title(tags[-1])
pylab.show()

In [ ]:
import numpy as np

#shuffle
indexes = np.arange(len(images))
np.random.shuffle(indexes)
images = torch.tensor(np.array(images)[indexes], dtype=torch.float32).cuda()
tags = torch.tensor(np.array(tags)[indexes], dtype=torch.float32).cuda()

In [ ]:
images.shape

In [ ]:
losses = []
acc = []

def sigmoid(x):
  # https://en.wikipedia.org/wiki/Sigmoid_function
  return 1/(1+torch.exp(-x))

m = Model().cuda()
adam  = torch.optim.Adam(m.parameters())

loss = torch.nn.BCELoss()
for EPOCH in range(30):
  for i in range(len(images)):
    op.zero_grad()
    # https://stackoverflow.com/questions/65470807/how-to-add-a-new-dimension-to-a-pytorch-tensor
    predicted = sigmoid(m(
        torch.unsqueeze(torch.permute(images[i], (2, 0, 1)), 0)
        ))
    # predicted = sigmoid(m(torch.permute(images[i], (2, 0, 1)).unsqueeze(0)))
    l = loss(predicted, torch.unsqueeze(torch.unsqueeze(tags[i], 0), 0).cuda())
    l.backward()
    adam.step()
    # adam.setp()
    losses.append(l)
    acc.append(torch.round(predicted)==tags[i])
    if i%100==0:
      print(loss)

In [ ]:
torch.unsqueeze(tags[i], 0).cuda().shape